# **Aprendizaje Automático No Supervisado por Asociación**

In [ ]:
#--------------------------------------------------------------------------------
# Paso 0. Kaggle API (opcional si decides usar la API)
#--------------------------------------------------------------------------------
# Instalamos las librerías mlxtend y kaggle
#--------------------------------------------------------------------------------
!pip install mlxtend --quiet
!pip install kaggle --quiet

#---------------------------------------------------------------------------------
# Cargamos las librerías necesarias
#---------------------------------------------------------------------------------
import zipfile

#--------------------------------------------------------------------------------
# Descargamos el dataset desde Kaggle (opcional si ya tienes el dataset descargado)
#--------------------------------------------------------------------------------
!kaggle datasets download -d uciml/pima-indians-diabetes-database --quiet

#--------------------------------------------------------------------------------
# Creamos la carpeta que va a contener el fichero descargado
#--------------------------------------------------------------------------------
!mkdir -p ~/.kaggle

#--------------------------------------------------------------------------------
# Extraemos los datos
#--------------------------------------------------------------------------------
with zipfile.ZipFile( "pima-indians-diabetes-database.zip", "r" ) as zip_ref:
    zip_ref.extractall( "dataset" )


Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
License(s): CC0-1.0


In [9]:
#---------------------------------------------------------------------------------
# Paso 1: Importamos de Bibliotecas
#---------------------------------------------------------------------------------
import pandas as pd
import warnings

from collections                import Counter
from imblearn.over_sampling     import SMOTE
from mlxtend.frequent_patterns  import apriori, association_rules

# Suprimimos las advertencias específicas de deprecación
warnings.filterwarnings( "ignore", category = DeprecationWarning )

#--------------------------------------------------------------------------------
# Paso 2. Cargamos los datos en un DataFrame
#--------------------------------------------------------------------------------
df = pd.read_csv( "dataset/diabetes.csv" )

#--------------------------------------------------------------------------------
# Paso 3. Seleccionamos variables relevantes para Apriori
#--------------------------------------------------------------------------------
df_bin = df[ [ 'Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction', 'Outcome' ] ].copy()

#--------------------------------------------------------------------------------
# Paso 4. Convertimos las variables a valores binarios (mantener como booleanos)
#--------------------------------------------------------------------------------
df_bin[ 'Glucose' ]                  = df_bin[ 'Glucose'                  ].apply(lambda x: 1 if x >= 140 else 0)
df_bin[ 'BMI' ]                      = df_bin[ 'BMI'                      ].apply(lambda x: 1 if x >=  30 else 0)
df_bin[ 'Age' ]                      = df_bin[ 'Age'                      ].apply(lambda x: 1 if x >=  50 else 0)
df_bin[ 'DiabetesPedigreeFunction' ] = df_bin[ 'DiabetesPedigreeFunction' ].apply(lambda x: 1 if x >= 0.5 else 0)
df_bin[ 'Outcome' ]                  = df_bin[ 'Outcome'                  ].apply(lambda x: 1 if x ==   1 else 0)

#--------------------------------------------------------------------------------
# Paso 5. Nivelamos las clases usando SMOTE
#   La técnica SMOTE ayuda a balancear el conjunto de datos para evitar que el modelo favorezca la clase mayoritaria
#--------------------------------------------------------------------------------
X = df_bin.drop( 'Outcome', axis = 1 )  # Eliminar la columna 'Outcome' de las características
y = df_bin[ 'Outcome' ]                 # La columna 'Outcome' es el objetivo

# Aplicamos SMOTE para sobremuestrear las clases
smote = SMOTE( random_state = 42 )
X_res, y_res = smote.fit_resample( X, y )

# Creamos el DataFrame balanceado
balanced_df = pd.DataFrame( X_res, columns = X.columns )
balanced_df[ 'Outcome' ] = y_res

#--------------------------------------------------------------------------------
# Paso 6. Generamos los conjuntos de elementos más frecuentes con Apriori
#--------------------------------------------------------------------------------
#   df            DataFrame con datos en formato binario
#   min_support   Umbral mínimo de soporte para considerar conjuntos frecuentes
#                   valores altos (0.3  a 0.5) generan pocas reglas pero muy relevantes
#                   valores bajos (0.05 a 0.1) generan muchas reglas pero algunas poco significativas o con ruido
#   use_colnames  Usa los nombres de las columnas en lugar de índices numéricos
#   max_len       Tamaño máximo del conjunto de items generados.
#   verbose       Muestra el progreso del algoritmo
#   low_memory    Usa menos memoria (útil ante grandes conjuntos de datos)
#   n_jobs        Número de procesadores en paralelo (-1 usa todos los disponibles) NO EXISTE EN mlxtend
#--------------------------------------------------------------------------------
conjuntoElementosFrecuentes = apriori(
    balanced_df.round().astype( int ), # aseguramos valores binarios
    min_support   = 0.2,
    max_len       = 5,
    verbose       = False,
    use_colnames  = True
    )

#--------------------------------------------------------------------------------
# Paso 7. Generamos las reglas de asociación con association_rules en función de los conjuntos de elementos más frecuentes
#--------------------------------------------------------------------------------
#   frequent_itemsets   Contiene los conjuntos de elementos frecuentes y su soporte
#   metric              Métricas que ayudan a evaluar la calidad de las reglas de asociación
#                           confidence    probabilidad de que al ser cierto el Antecedente también lo sea el Consequente
#                             cerca de 1    Fuerte relación entre Antecedente y Consecuente
#                             valor bajo    Antecedente ocurre muchas veces sin que ocurra Consecuente
#                           lift          probabilidad de que el Consecuente ocurra si el Antecedente es cierto, en comparación con la probabilidad general del Consecuente
#                             > 1   Antecedente y Consecuente positivamente correlacionados
#                             = 1   Antecedente y Consecuente son independientes
#                             < 1   Antecedente reduce la probabilidad de Consecuente
#                           support       mide la frecuencia con la que ocurre un conjunto de valores en el dataset
#                             > 0.5   Regla más común y confiable
#                             < 0.5   Puede ser un patrón raro pero valioso
#                           leverage      mide la frecuencia de que Antecedente y Consecuente ocurran juntos en comparación de si fueran independientes
#                             > 0   Antecedente y Consecuente aparecen juntos más de lo esperado
#                             = 0   NO hay relación
#                             < 0   Antecedente y Consecuente aparecen juntos menos de lo esperado
#                           conviction    mide la dependencia del Consecuente frente al Antecedente comparado con que ocurra el Consecuente por sí solo
#                             > 1           Relación fuerte en Antecedente y Consecuente
#                             cercanos a 1  Relación débil
#   min_threshold       Mantiene reglas con el nivel de la métrica especificado
#--------------------------------------------------------------------------------
#Reglas = association_rules(
#    conjuntoElementosFrecuentes,
#    metric        = "confidence",
#    min_threshold = 0.2 )

# Generamos ahora todas las reglas
Reglas = association_rules( conjuntoElementosFrecuentes )

#--------------------------------------------------------------------------------
# Paso 8. Filtramos por las reglas más importantes:
#     confidence > 0.2
#     lift       > 1.2
#--------------------------------------------------------------------------------
# Filtramos las reglas que cumplen con los valores de confidence y lift
#reglasFiltradas = Reglas[ ( Reglas[ 'confidence' ] > 0.2 ) & ( Reglas[ 'lift' ] > 1.2 ) ]
reglasFiltradas = Reglas[ ( Reglas[ 'confidence' ] > 0.150 ) ]

#--------------------------------------------------------------------------------
# Paso 9. Filtramos ahora sólo por las columnas necesarias
#--------------------------------------------------------------------------------
reglasFiltradas = reglasFiltradas[ [ 'antecedents', 'consequents', 'support', 'confidence', 'lift' ] ].copy()

#--------------------------------------------------------------------------------
# Paso 10. Convertimos antecedentes y consecuentes al formato deseado
#--------------------------------------------------------------------------------
reglasFiltradas[ 'antecedents' ] = reglasFiltradas[ 'antecedents' ].apply( lambda x: ', '.join( [ str( item ) for item in x ] ) )
reglasFiltradas[ 'consequents' ] = reglasFiltradas[ 'consequents' ].apply( lambda x: ', '.join( [ str( item ) for item in x ] ) )

#--------------------------------------------------------------------------------
# Paso 11. Calculamos el tamaño máximo de los valores en 'antecedents' y 'consequents'
#--------------------------------------------------------------------------------
maxAntecedents, maxConsequents = 16, 11
if not reglasFiltradas.empty:
    maxAntecedents = max( max( reglasFiltradas[ 'antecedents' ].apply( len ) ), 16 )
    maxConsequents = max( max( reglasFiltradas[ 'consequents' ].apply( len ) ), 11 )

#--------------------------------------------------------------------------------
# Paso 12. Filtramos las reglas donde "Consequents" sólo contenga "Outcome"
#--------------------------------------------------------------------------------
# Verificamos si "Outcome" es el único valor en "Consequents"
reglasFiltradas[ 'only_outcome' ] = reglasFiltradas[ 'consequents' ].apply(lambda x: x == "Outcome" )

#--------------------------------------------------------------------------------
# Paso 13. Ordenamos las reglas para que las que contienen sólo "Outcome" y en "Consequents" aparezcan al final
#--------------------------------------------------------------------------------
reglasFiltradasOrdenadas = reglasFiltradas.sort_values( by = 'only_outcome', ascending = True )

# Eliminar la columna "only_outcome" ya que no es necesaria para la visualización
reglasFiltradasOrdenadas = reglasFiltradasOrdenadas.drop( columns = [ 'only_outcome' ] )

#--------------------------------------------------------------------------------
# Paso 14. Mostramos los encabezados con tamaños dinámicos basados en el tamaño máximo de las listas
#--------------------------------------------------------------------------------
print( "\nReglas de asociación filtradas y ordenadas:\n" )
print( f"{'Antecedentes':<{maxAntecedents}}     {'Consecuencias':<{maxConsequents}}   {'Soporte':<10}{'Confianza':<12}{'Fuerza de asociación':<10}")
print( "-" * (maxAntecedents + maxConsequents + 50 ) )

#--------------------------------------------------------------------------------
# Paso 15. Imprimimos las reglas ordenadas con los valores correspondientes
#--------------------------------------------------------------------------------
if reglasFiltradasOrdenadas.empty:
  print( "No se ha encontrado ninguna regla que cumpla con los criterios." )
else:
  for index, row in reglasFiltradasOrdenadas.iterrows():
    print( f"( {row[ 'antecedents' ]:<{maxAntecedents}}) → ({row['consequents']:<{maxConsequents}})    {row['support']*100:.2f}%     {row['confidence']*100:.2f}%           {row['lift']:.2f}" )



Reglas de asociación filtradas y ordenadas:

Antecedentes                          Consecuencias   Soporte   Confianza   Fuerza de asociación
----------------------------------------------------------------------------------------------
( Glucose                          ) → (BMI        )    25.50%     81.99%           1.25
( Outcome                          ) → (BMI        )    40.40%     80.80%           1.23
( Glucose, Outcome                 ) → (BMI        )    21.80%     87.55%           1.33
( DiabetesPedigreeFunction, Outcome) → (BMI        )    21.00%     86.78%           1.32
( Glucose                          ) → (Outcome    )    24.90%     80.06%           1.60
( Glucose, BMI                     ) → (Outcome    )    21.80%     85.49%           1.71


In [ ]:
#---------------------------------------------------------------------------------
# Paso 1: Importamos de Bibliotecas
#---------------------------------------------------------------------------------
import pandas as pd
import warnings

from collections                import Counter
from imblearn.over_sampling     import SMOTE
from mlxtend.frequent_patterns  import apriori, association_rules

# Suprimimos las advertencias específicas de deprecación
warnings.filterwarnings( "ignore", category = DeprecationWarning )

#--------------------------------------------------------------------------------
# Paso 2. Cargamos los datos en un DataFrame
#--------------------------------------------------------------------------------
df = pd.read_csv( "dataset/diabetes.csv" )

#--------------------------------------------------------------------------------
# Paso 3. Seleccionamos variables relevantes para Apriori
#--------------------------------------------------------------------------------
df_bin = df[ [ 'Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction', 'Outcome' ] ].copy()

#--------------------------------------------------------------------------------
# Paso 4. Convertimos las variables a valores binarios (mantener como booleanos)
#--------------------------------------------------------------------------------
df_bin[ 'Glucose' ]                  = df_bin[ 'Glucose'                  ].apply(lambda x: 1 if x >= 140 else 0)
df_bin[ 'BMI' ]                      = df_bin[ 'BMI'                      ].apply(lambda x: 1 if x >=  30 else 0)
df_bin[ 'Age' ]                      = df_bin[ 'Age'                      ].apply(lambda x: 1 if x >=  50 else 0)
df_bin[ 'DiabetesPedigreeFunction' ] = df_bin[ 'DiabetesPedigreeFunction' ].apply(lambda x: 1 if x >= 0.5 else 0)
df_bin[ 'Outcome' ]                  = df_bin[ 'Outcome'                  ].apply(lambda x: 1 if x ==   1 else 0)

#--------------------------------------------------------------------------------
# Paso 5. Nivelamos las clases usando SMOTE
#   La técnica SMOTE ayuda a balancear el conjunto de datos para evitar que el modelo favorezca la clase mayoritaria
#--------------------------------------------------------------------------------
X = df_bin.drop( 'Outcome', axis = 1 )  # Eliminar la columna 'Outcome' de las características
y = df_bin[ 'Outcome' ]                 # La columna 'Outcome' es el objetivo

# Aplicamos SMOTE para sobremuestrear las clases
smote = SMOTE( random_state = 42 )
X_res, y_res = smote.fit_resample( X, y )

# Creamos el DataFrame balanceado
balanced_df = pd.DataFrame( X_res, columns = X.columns )
balanced_df[ 'Outcome' ] = y_res

#--------------------------------------------------------------------------------
# Paso 6. Generamos los conjuntos de elementos más frecuentes con Apriori
#--------------------------------------------------------------------------------
#   df            DataFrame con datos en formato binario
#   min_support   Umbral mínimo de soporte para considerar conjuntos frecuentes
#                   valores altos (0.3  a 0.5) generan pocas reglas pero muy relevantes
#                   valores bajos (0.05 a 0.1) generan muchas reglas pero algunas poco significativas o con ruido
#   use_colnames  Usa los nombres de las columnas en lugar de índices numéricos
#   max_len       Tamaño máximo del conjunto de items generados.
#   verbose       Muestra el progreso del algoritmo
#   low_memory    Usa menos memoria (útil ante grandes conjuntos de datos)
#   n_jobs        Número de procesadores en paralelo (-1 usa todos los disponibles) NO EXISTE EN mlxtend
#--------------------------------------------------------------------------------
conjuntoElementosFrecuentes = apriori(
    balanced_df.round().astype( int ), # aseguramos valores binarios
    min_support   = 0.1,
    max_len       = 3,
    verbose       = False,
    use_colnames  = True
    )

#--------------------------------------------------------------------------------
# Paso 7. Generamos las reglas de asociación con association_rules en función de los conjuntos de elementos más frecuentes
#--------------------------------------------------------------------------------
#   frequent_itemsets   Contiene los conjuntos de elementos frecuentes y su soporte
#   metric              Métricas que ayudan a evaluar la calidad de las reglas de asociación
#                           confidence    probabilidad de que al ser cierto el Antecedente también lo sea el Consequente
#                             cerca de 1    Fuerte relación entre Antecedente y Consecuente
#                             valor bajo    Antecedente ocurre muchas veces sin que ocurra Consecuente
#                           lift          probabilidad de que el Consecuente ocurra si el Antecedente es cierto, en comparación con la probabilidad general del Consecuente
#                             > 1   Antecedente y Consecuente positivamente correlacionados
#                             = 1   Antecedente y Consecuente son independientes
#                             < 1   Antecedente reduce la probabilidad de Consecuente
#                           support       mide la frecuencia con la que ocurre un conjunto de valores en el dataset
#                             > 0.5   Regla más común y confiable
#                             < 0.5   Puede ser un patrón raro pero valioso
#                           leverage      mide la frecuencia de que Antecedente y Consecuente ocurran juntos en comparación de si fueran independientes
#                             > 0   Antecedente y Consecuente aparecen juntos más de lo esperado
#                             = 0   NO hay relación
#                             < 0   Antecedente y Consecuente aparecen juntos menos de lo esperado
#                           conviction    mide la dependencia del Consecuente frente al Antecedente comparado con que ocurra el Consecuente por sí solo
#                             > 1           Relación fuerte en Antecedente y Consecuente
#                             cercanos a 1  Relación débil
#   min_threshold       Mantiene reglas con el nivel de la métrica especificado
#--------------------------------------------------------------------------------
#Reglas = association_rules(
#    conjuntoElementosFrecuentes,
#    metric        = "confidence",
#    min_threshold = 0.2 )

# Generamos ahora todas las reglas
Reglas = association_rules( conjuntoElementosFrecuentes,
   metric        = "confidence",
   min_threshold = 0.6)

#--------------------------------------------------------------------------------
# Paso 8. Filtramos por las reglas más importantes:
#     confidence > 0.2
#     lift       > 1.2
#--------------------------------------------------------------------------------
# Filtramos las reglas que cumplen con los valores de confidence y lift
#reglasFiltradas = Reglas[ ( Reglas[ 'confidence' ] > 0.2 ) & ( Reglas[ 'lift' ] > 1.2 ) ]
reglasFiltradas = Reglas[ ( Reglas[ 'confidence' ] > 0.80 ) ]

#--------------------------------------------------------------------------------
# Paso 9. Filtramos ahora sólo por las columnas necesarias
#--------------------------------------------------------------------------------
reglasFiltradas = reglasFiltradas[ [ 'antecedents', 'consequents', 'support', 'confidence', 'lift' ] ].copy()

#--------------------------------------------------------------------------------
# Paso 10. Convertimos antecedentes y consecuentes al formato deseado
#--------------------------------------------------------------------------------
reglasFiltradas[ 'antecedents' ] = reglasFiltradas[ 'antecedents' ].apply( lambda x: ', '.join( [ str( item ) for item in x ] ) )
reglasFiltradas[ 'consequents' ] = reglasFiltradas[ 'consequents' ].apply( lambda x: ', '.join( [ str( item ) for item in x ] ) )

#--------------------------------------------------------------------------------
# Paso 11. Calculamos el tamaño máximo de los valores en 'antecedents' y 'consequents'
#--------------------------------------------------------------------------------
maxAntecedents, maxConsequents = 16, 11
if not reglasFiltradas.empty:
    maxAntecedents = max( max( reglasFiltradas[ 'antecedents' ].apply( len ) ), 16 )
    maxConsequents = max( max( reglasFiltradas[ 'consequents' ].apply( len ) ), 11 )

#--------------------------------------------------------------------------------
# Paso 12. Filtramos las reglas donde "Consequents" sólo contenga "Outcome"
#--------------------------------------------------------------------------------
# Verificamos si "Outcome" es el único valor en "Consequents"
reglasFiltradas[ 'only_outcome' ] = reglasFiltradas[ 'consequents' ].apply(lambda x: x == "Outcome" )

#--------------------------------------------------------------------------------
# Paso 13. Ordenamos las reglas para que las que contienen sólo "Outcome" y en "Consequents" aparezcan al final
#--------------------------------------------------------------------------------
reglasFiltradasOrdenadas = reglasFiltradas.sort_values( by = 'only_outcome', ascending = True )

# Eliminar la columna "only_outcome" ya que no es necesaria para la visualización
reglasFiltradasOrdenadas = reglasFiltradasOrdenadas.drop( columns = [ 'only_outcome' ] )

#--------------------------------------------------------------------------------
# Paso 14. Mostramos los encabezados con tamaños dinámicos basados en el tamaño máximo de las listas
#--------------------------------------------------------------------------------
print( "\nReglas de asociación filtradas y ordenadas:\n" )
print( f"{'Antecedentes':<{maxAntecedents}}     {'Consecuencias':<{maxConsequents}}   {'Soporte':<10}{'Confianza':<12}{'Fuerza de asociación':<10}")
print( "-" * (maxAntecedents + maxConsequents + 50 ) )


#-------------------------------------------------------------------------------
# Paso 15. Imprimimos las reglas ordenadas con los valores correspondientes
#-------------------------------------------------------------------------------
if reglasFiltradasOrdenadas.empty:
  print( "No se ha encontrado ninguna regla que cumpla con los criterios." )
else:
  for index, row in reglasFiltradasOrdenadas.iterrows():
    print( f"( {row[ 'antecedents' ]:<{maxAntecedents}}) → ({row['consequents']:<{maxConsequents}})    {row['support']*100:.2f}%     {row['confidence']*100:.2f}%           {row['lift']:.2f}" )



Reglas de asociación filtradas y ordenadas:

Antecedentes                          Consecuencias   Soporte   Confianza   Fuerza de asociación
----------------------------------------------------------------------------------------------
( Glucose                          ) → (BMI        )    25.50%     81.99%           1.25
( Outcome                          ) → (BMI        )    40.40%     80.80%           1.23
( Glucose, DiabetesPedigreeFunction) → (BMI        )    13.00%     90.91%           1.38
( Glucose, Outcome                 ) → (BMI        )    21.80%     87.55%           1.33
( DiabetesPedigreeFunction, Outcome) → (BMI        )    21.00%     86.78%           1.32
( Glucose                          ) → (Outcome    )    24.90%     80.06%           1.60
( Glucose, BMI                     ) → (Outcome    )    21.80%     85.49%           1.71
( Glucose, DiabetesPedigreeFunction) → (Outcome    )    12.00%     83.92%           1.68
